<a href="https://colab.research.google.com/github/AishwaryaVelumani/Python-IP-ML/blob/master/LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
import numpy as np
import pandas as pd
import lightgbm
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.metrics import accuracy_score

In [2]:
from google.colab import files
uploaded= files.upload()

Saving Adults.csv to Adults.csv


In [73]:
ds= pd.read_csv('Adults.csv')

In [53]:
ds.head(10)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
6,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K


In [54]:
le= LabelEncoder()
le.fit(ds.income)
ds.income= Series(le.transform(ds.income))
ds.income.value_counts()
#type(ds.income)

0    37155
1    11687
Name: income, dtype: int64

In [55]:
#oh- one_hot
oh_workclass= pd.get_dummies(ds['workclass'])
oh_edu= pd.get_dummies(ds['education'])
oh_mari= pd.get_dummies(ds['marital-status'])
oh_occu= pd.get_dummies(ds['occupation'])
oh_rela= pd.get_dummies(ds['relationship'])
oh_race= pd.get_dummies(ds['race'])
oh_gen= pd.get_dummies(ds['gender'])
oh_nati= pd.get_dummies(ds['native-country'])

In [56]:
ds.drop(labels=['workclass','education','marital-status','occupation','relationship','race','gender','native-country'],axis=1,inplace=True)

In [57]:
ds.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,income
0,25,226802,7,0,0,40,0
1,38,89814,9,0,0,50,0
2,28,336951,12,0,0,40,1
3,44,160323,10,7688,0,40,1
4,18,103497,10,0,0,30,0


In [58]:
ds= pd.concat([ds,oh_workclass,oh_edu,oh_mari,oh_occu,oh_rela,oh_race,oh_gen,oh_nati],axis=1)

In [59]:
ds.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,income,?,Federal-gov,Local-gov,Never-worked,Private,Self-emp-inc,Self-emp-not-inc,State-gov,Without-pay,10th,11th,12th,1st-4th,5th-6th,7th-8th,9th,Assoc-acdm,Assoc-voc,Bachelors,Doctorate,HS-grad,Masters,Preschool,Prof-school,Some-college,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,?,...,Canada,China,Columbia,Cuba,Dominican-Republic,Ecuador,El-Salvador,England,France,Germany,Greece,Guatemala,Haiti,Holand-Netherlands,Honduras,Hong,Hungary,India,Iran,Ireland,Italy,Jamaica,Japan,Laos,Mexico,Nicaragua,Outlying-US(Guam-USVI-etc),Peru,Philippines,Poland,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
0,25,226802,7,0,0,40,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,0,0,50,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,28,336951,12,0,0,40,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,44,160323,10,7688,0,40,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,18,103497,10,0,0,30,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [60]:
ds.columns

Index(['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'income', '?', 'Federal-gov', 'Local-gov',
       ...
       'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand',
       'Trinadad&Tobago', 'United-States', 'Vietnam', 'Yugoslavia'],
      dtype='object', length=109)

In [61]:
_,i= np.unique(ds.columns,return_index=True)
print(_)
print()
print(i)

['10th' '11th' '12th' '1st-4th' '5th-6th' '7th-8th' '9th' '?'
 'Adm-clerical' 'Amer-Indian-Eskimo' 'Armed-Forces' 'Asian-Pac-Islander'
 'Assoc-acdm' 'Assoc-voc' 'Bachelors' 'Black' 'Cambodia' 'Canada' 'China'
 'Columbia' 'Craft-repair' 'Cuba' 'Divorced' 'Doctorate'
 'Dominican-Republic' 'Ecuador' 'El-Salvador' 'England' 'Exec-managerial'
 'Farming-fishing' 'Federal-gov' 'Female' 'France' 'Germany' 'Greece'
 'Guatemala' 'HS-grad' 'Haiti' 'Handlers-cleaners' 'Holand-Netherlands'
 'Honduras' 'Hong' 'Hungary' 'Husband' 'India' 'Iran' 'Ireland' 'Italy'
 'Jamaica' 'Japan' 'Laos' 'Local-gov' 'Machine-op-inspct' 'Male'
 'Married-AF-spouse' 'Married-civ-spouse' 'Married-spouse-absent'
 'Masters' 'Mexico' 'Never-married' 'Never-worked' 'Nicaragua'
 'Not-in-family' 'Other' 'Other-relative' 'Other-service'
 'Outlying-US(Guam-USVI-etc)' 'Own-child' 'Peru' 'Philippines' 'Poland'
 'Portugal' 'Preschool' 'Priv-house-serv' 'Private' 'Prof-school'
 'Prof-specialty' 'Protective-serv' 'Puerto-Rico' 'Sales

In [62]:
ds= ds.iloc[:,i]

In [63]:
#dont make inplace as true else income gets removed from ds
x=ds.drop('income',axis=1)

In [64]:
ds.head()

,10th,11th,12th,1st-4th,5th-6th,7th-8th,9th,?,Adm-clerical,Amer-Indian-Eskimo,Armed-Forces,Asian-Pac-Islander,Assoc-acdm,Assoc-voc,Bachelors,Black,Cambodia,Canada,China,Columbia,Craft-repair,Cuba,Divorced,Doctorate,Dominican-Republic,Ecuador,El-Salvador,England,Exec-managerial,Farming-fishing,Federal-gov,Female,France,Germany,Greece,Guatemala,HS-grad,Haiti,Handlers-cleaners,Holand-Netherlands,...,Own-child,Peru,Philippines,Poland,Portugal,Preschool,Priv-house-serv,Private,Prof-school,Prof-specialty,Protective-serv,Puerto-Rico,Sales,Scotland,Self-emp-inc,Self-emp-not-inc,Separated,Some-college,South,State-gov,Taiwan,Tech-support,Thailand,Transport-moving,Trinadad&Tobago,United-States,Unmarried,Vietnam,White,Widowed,Wife,Without-pay,Yugoslavia,age,capital-gain,capital-loss,educational-num,fnlwgt,hours-per-week,income
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,25,0,0,7,226802,40,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,38,0,0,9,89814,50,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,28,0,0,12,336951,40,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,44,7688,0,10,160323,40,1
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,18,0,0,10,103497,30,0


In [65]:
y=ds.income

In [67]:
y.isnull().sum()

0

In [68]:
x.isnull().sum()

10th               0
11th               0
12th               0
1st-4th            0
5th-6th            0
                  ..
capital-gain       0
capital-loss       0
educational-num    0
fnlwgt             0
hours-per-week     0
Length: 106, dtype: int64

In [90]:
print(x.shape)
print(y.shape)

(48842, 106)
(48842,)


In [93]:
xtr,xte,ytr,yte= train_test_split(x,y,test_size=0.3,random_state=0)

In [94]:
print(xtr.shape)
print(ytr.shape)


(34189, 106)
(34189,)


In [104]:
start= datetime.now()
lgb= lightgbm.LGBMClassifier(num_leaves=50,max_depth=4,learning_rate=0.05,objective='binary',random_state=0,max_bin=200)
lgb.fit(xtr,ytr)
stop= datetime.now()
ypr= lgb.predict(xte)
print('Accuracy score: {:0.4f}'.format(100*accuracy_score(ypr,yte)),'%')
print('Execution time:',stop-start)

Accuracy score: 86.3441 %
Execution time: 0:00:00.497673


In [103]:
print(datetime.now())

2020-07-28 12:01:42.302431
